In [3]:
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping
import numpy as np
from sklearn import cross_validation
from keras_input_data import make_idx_data
from load_vai import loadVAI
import _pickle as cPickle
from metrics import continuous_metrics
from keras.preprocessing.text import Tokenizer
import keras
import gensim
from sklearn.kernel_ridge import KernelRidge
from gensim.models.doc2vec import Doc2Vec , TaggedDocument
from sentences_getvec import getSentences_vec
from sklearn import datasets, linear_model
from sklearn.svm import SVR


if __name__ == '__main__':
    x = cPickle.load(open("mr.p", "rb"))
    revs, W, W2, word_idx_map, vocab = x[0], x[1], x[2], x[3], x[4]
    print("data loaded!")
    sentences=[]
    for rev in revs:
        sentence = rev['text']
        sentences.append(sentence)
    idx_data = make_idx_data(sentences, word_idx_map)
    #print(idx_data)

    dim = 'I'
    column = loadVAI(dim)
    irony=column
    maxlen = 87  # cut texts after this number of words (among top max_features most common words)
    batch_size = 8

    option = 'Irony'  # or Arousal,irony
    Y = np.array(irony)
    Y = [float(x) for x in Y]
    print(option + ' prediction.......................')

    # X_train, X_test, y_train, y_test = cross_validation.train_test_split(idx_data, Y, test_size=0.2,
    #                                                                      random_state=2)
    n_MAE=0
    n_Pearson_r=0
    n_Spearman_r=0
    n_MSE=0
    n_R2=0
    n_MSE_sqrt=0
    SEED = 42

    ID=list(range(1,1005))
    n = 5  # repeat the CV procedure 5 times to get more precise results
    
    for i in range(n):
        # for each iteration, randomly hold out 20% of the data as CV set
        X_train, X_test,ID_train,ID_test, y_train, y_test = cross_validation.train_test_split(
            idx_data,ID, Y, test_size=.20, random_state=i * SEED)
        print(ID_train)
        print("＿＿＿＿＿＿＿＿＿＿＿＿")
        print(ID_test)

        print(len(X_train), 'train sequences')
        print(len(X_test), 'test sequences')
        print("Pad sequences (samples x time)")
        X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
        X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
        print('X_train shape:', X_train.shape)
        print('X_test shape:', X_test.shape)
        #print(X_train)
        X_train=np.array(getSentences_vec(X_train))
        print(X_train.shape)
        X_test =np.array(getSentences_vec(X_test))
        print(X_test.shape)
        """
        convert 3dim to 2dim
        """
        nsamples, nx, ny = X_train.shape
        X_train = X_train.reshape((nsamples, nx * ny))
        nsamples, nx, ny = X_test.shape
        X_test= X_test.reshape((nsamples, nx * ny))

        #print(X_test)

        print("")
        #print("Method = Linear ridge regression with doc2vec features")
        # model = KernelRidge(kernel='linear')
        # model.fit(X_train, y_train)
        #model = KernelRidge(kernel='linear')
        #model=linear_model.LinearRegression()
        model = SVR(kernel='poly',C=1e3, epsilon=0.2,gamma=0.1)
        # print(X_train.shape)
        # print(len(y_train))
        #model.fit(X_train, y_train)
        results = model.fit(X_train, y_train).predict(X_test)
        #results = model.predict(X_test)
        # print('Y_test: %s' % str(y_test))
        # print('Predict value: %s' % str(predict))
        #print(y_test)
        #print(results)
        estimate=continuous_metrics(y_test, results, 'prediction result:')
        # MSE, MAE, Pearson_r, R2, Spearman_r, MSE_sqrt
    
        n_MAE += estimate[0]
        n_Pearson_r += estimate[1]
       
    ndigit=3
   
    avg_MAE =  round(n_MAE/5, ndigit)
    avg_Pearson_r =  round(n_Pearson_r/5, ndigit)
   
    print('average evaluate result:')
    print(avg_MAE ,avg_Pearson_r)

data loaded!
------------- now is irony------------
Irony prediction.......................
[466, 400, 921, 820, 699, 413, 519, 521, 886, 114, 615, 38, 687, 899, 352, 159, 155, 41, 410, 539, 859, 774, 267, 409, 295, 666, 525, 19, 472, 530, 309, 984, 448, 63, 80, 683, 194, 776, 182, 998, 584, 201, 912, 996, 714, 407, 3, 675, 444, 254, 578, 86, 441, 76, 838, 898, 468, 925, 39, 949, 763, 919, 141, 821, 793, 609, 980, 494, 419, 1003, 870, 51, 163, 887, 597, 655, 527, 818, 173, 662, 331, 830, 860, 769, 40, 565, 379, 157, 223, 644, 211, 749, 965, 703, 168, 562, 475, 417, 11, 106, 69, 482, 955, 437, 259, 569, 588, 305, 814, 943, 929, 479, 21, 959, 772, 423, 286, 72, 50, 180, 404, 619, 117, 294, 77, 49, 359, 264, 811, 765, 671, 333, 65, 801, 53, 928, 125, 653, 628, 631, 957, 480, 119, 13, 158, 338, 128, 47, 236, 673, 242, 751, 997, 964, 966, 502, 326, 903, 785, 351, 365, 18, 187, 507, 339, 207, 627, 346, 314, 243, 433, 171, 188, 905, 134, 302, 734, 593, 319, 79, 524, 347, 199, 868, 492, 920, 5